# <h1><center>Estructuras de datos</center></h1>
## <center>"Extracción de datos de internet con Scrapy y Python"</center>
#### Universidad Nacional de Tres de Febrero, noviembre 2020.
##### Jeremías Mazzetti
##### Consultar documentación de Scrapy [aquí](https://docs.scrapy.org/en/latest/)


### <center>Módulos requeridos.</center>

In [1]:
import csv
import operator
import string
import unicodedata
from abc import ABC, abstractmethod
from datetime import datetime
from scrapy.crawler import CrawlerProcess
from scrapy.spiders import Rule,CrawlSpider
from scrapy.linkextractors import LinkExtractor
from nltk.corpus import stopwords
from scrapy import Item, Field
from scrapy.loader import ItemLoader

### <center>Campos de interés.</center>
Se utilizó la clase Item incluida en Scrapy ya que permite constuir facilmente objetos de diccionarios que luego son utiles a la hora de hacer el yield y guardar en json o csv sin tener que importar módulos externos.
Cada Field() representa la clave de cada diccionario contenido en el Item.

In [2]:
class Producto(Item):
    supermercado = Field()
    marca=Field()
    producto=Field()
    categoria=Field()
    precio=Field()
    link=Field()
    hora=Field()
    fecha_de_extraccion=Field()

### <center> Formato y remoción de stopwords.</center>
Se utilizó el módulo nltk para quitar stopwords y se importó el módulo string incluido en Python para quitar caracteres indeseados, que podría arruinar la búsqueda. Éstas funciones se copiaron aquí porque de estas depende los resultados que se obtendran. Las mismas forman parte de los métodos de algunos de los Buscadores.

In [3]:
def formatear(busqueda):
    busqueda = ''.join((c for c in unicodedata.normalize('NFD', busqueda) if unicodedata.category(c) != 'Mn'))
    for char in string.punctuation+"¡¿":
        busqueda=busqueda.replace(char,'')
    return busqueda.lower()

def quitar_stopwords(busqueda):
    palabras_busqueda = busqueda.split(' ')
    busqueda_sin_stopwords = ''
    for palabras in palabras_busqueda:
        if palabras not in stopwords.words('spanish'):
            busqueda_sin_stopwords += palabras + ' '
    return busqueda_sin_stopwords[:-1]

### <center> Buscadores.</center>
Para determinar como construir las clases del tipo Buscador se tuvo en cuanta que atributos y métodos son comunes a todos los tipos de Buscadores, cuales son abstractos y se deben definir en cada tipo de Buscador en particular.

A continuación podrás apreciar la clase Buscador que es abstracta ya que contiene el método generar_url_reglas que debe estar contenido en las clases hijas pero que su comportamiento debe cambiar de acuerdo a el tipo de busqueda que el usuario requiere.La misma cuanta con los settings y método de ordenamiento que se utilizará luego de la extracción de datos, si el usuario así lo desea.

El metodo generar archivo generar_archivo_ordenado_por_menor_precio contempla dos tipos de fallas, si el archivo que requiere leer no exite y si hay incompatibilidad entre los datos que se estan intentado ordenar.

A modo de poder simplificar, se incluyen las clases requeridas y solo una spider

In [4]:
class Buscador(ABC):

    def __init__(self, busqueda, formato):
        self.busqueda = self.formatear(busqueda)
        self.tipo_busqueda = ''
        self.date = str(datetime.now().date())
        self.ruta_destino = 'Salida/' + self.busqueda.replace(' ', '_') + '_' + self.date + '.' + formato
        self.formato = formato
        self.process = CrawlerProcess({
            'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                          'Chrome/86.0.4240.111 Safari/537.36',
            'FEED_EXPORT_FIELDS': ['fecha_de_extraccion', 'precio', 'producto', 'marca', 'supermercado', 'categoria',
                                   'link', 'hora'],
            'FEED_EXPORT_ENCODING': 'utf-8',
            'FEED_FORMAT': formato,
            'FEED_URI': self.ruta_destino
        })

    def iniciar_busqueda(self):
        urls, reglas = self.generar_url_reglas()
        laamistad_start = urls[0]
        laamistad_rules = reglas[0]
        self.process.crawl(LaamistadSpider, start_urls=laamistad_start, rules=laamistad_rules)
        self.process.start()

    @abstractmethod
    def generar_url_reglas(self):
        pass

    def generar_archivo_ordenado_por_menor_precio(self):
        try:
            if self.formato == 'csv':
                with open(self.ruta_destino, 'r') as file:
                    reader = csv.DictReader(file)
                    unsorted_dict = {}
                    for fila in reader:
                        supermercado = fila['supermercado']
                        marca = fila['marca']
                        producto = fila['producto']
                        categoria = fila['categoria']
                        hora = fila['hora']
                        precio = fila['precio']
                        link = fila['link']
                        fecha_de_extraccion = fila['fecha_de_extraccion']
                        lista = [fecha_de_extraccion, producto, marca, supermercado, categoria, link, hora]
                        precio = precio.replace(" ", "").replace("$", "").replace(",", ".")
                        if precio != 'precio' and precio != 'xkg' and precio != '' and precio.count('.') == 1:
                            precio = float(precio)
                            if precio not in unsorted_dict.keys():
                                unsorted_dict[precio] = [lista]
                            else:
                                list_aux = []
                                for lista_rec in unsorted_dict.get(precio):
                                    list_aux.append(lista_rec)
                                list_aux.append(lista)
                                unsorted_dict[precio] = list_aux
                    sorted_dict = dict(sorted(unsorted_dict.items(), key=operator.itemgetter(0)))
                    with open(self.ruta_destino, 'w', newline="") as archivo:
                        campos_archivo = ['fecha_de_extraccion', 'precio', 'producto', 'marca', 'supermercado',
                                          'categoria', 'link', 'hora']
                        writer = csv.DictWriter(archivo, fieldnames=campos_archivo)
                        writer.writeheader()
                        for precio in sorted_dict.keys():
                            for lista in sorted_dict.get(precio):
                                writer.writerow({campos_archivo[0]: lista[0], campos_archivo[1]: precio,
                                                 campos_archivo[2]: lista[1],
                                                 campos_archivo[3]: lista[2], campos_archivo[4]: lista[3],
                                                 campos_archivo[5]: lista[4],
                                                 campos_archivo[6]: lista[5], campos_archivo[7]: lista[6]})
        except FileNotFoundError as e:
            print('No se encontró el archivo.')
        except ValueError as e:
            print('Hubo incompatibilidad datos.\nPuede que la pagina haya cambiado.')

    def formatear(self, busqueda):
        busqueda = ''.join((c for c in unicodedata.normalize('NFD', busqueda) if unicodedata.category(c) != 'Mn'))
        for char in string.punctuation + "¡¿":
            busqueda = busqueda.replace(char, '')
        return busqueda.lower()
    
class BuscadorVariable(Buscador):

    def __init__(self, busqueda, formato):
        super(BuscadorVariable, self).__init__(busqueda, formato)
        self.tipo_busqueda = 'variable'

    def generar_url_reglas(self):
        LaamistadSpider.busqueda = self.quitar_stopwords(self.busqueda)
        self.busqueda = self.quitar_stopwords(self.busqueda)
        laamistad_start = []
        laamistad_rules = [Rule(LinkExtractor(allow=r'page/')),
                           Rule(LinkExtractor(allow=r'tienda/'), callback=self.tipo_busqueda)]
        for palabra in self.busqueda.split(" "):
            laamistad_start.append('https://www.tiendalaamistad.com.ar/?s=' + palabra)
        urls = [laamistad_start]
        reglas = [laamistad_rules]
        return urls, reglas

    def quitar_stopwords(self, busqueda):
        palabras_busqueda = busqueda.split(' ')
        busqueda_sin_stopwords = ''
        for palabras in palabras_busqueda:
            if palabras not in stopwords.words('spanish'):
                busqueda_sin_stopwords += palabras + ' '
        return busqueda_sin_stopwords[:-1]
    
class BuscadorTodo(BuscadorVariable):

    def __init__(self, busqueda, formato):
        super(BuscadorTodo, self).__init__(busqueda, formato)
        self.tipo_busqueda = 'todo'

    def generar_url_reglas(self):
        LaamistadSpider.busqueda = self.quitar_stopwords(self.busqueda)
        self.busqueda = self.busqueda.split(' ')[0]
        laamistad_start = []
        laamistad_rules = [Rule(LinkExtractor(allow=r'page/')),
                           Rule(LinkExtractor(allow=r'tienda/'), callback=self.tipo_busqueda)]
        laamistad_start.append('https://www.tiendalaamistad.com.ar/?s=' + self.busqueda)
        urls = [laamistad_start]
        reglas = [laamistad_rules]
        return urls, reglas
    
class BuscadorExacto(Buscador):

    def __init__(self, busqueda, formato):
        super(BuscadorExacto, self).__init__(busqueda, formato)
        self.tipo_busqueda='exacto'

    def generar_url_reglas(self):
        LaamistadSpider.busqueda = self.busqueda
        laamistad_start = ['https://www.tiendalaamistad.com.ar/?s=' + self.busqueda.replace(' ', '+')]
        laamistad_rules = [Rule(LinkExtractor(allow=r'page/')),
                           Rule(LinkExtractor(allow=r'tienda/'), callback=self.tipo_busqueda)]
        urls = [laamistad_start]
        reglas = [laamistad_rules]
        return urls, reglas


### <center>Spider.</center>
En esta sección se encuentran uno de las spiders que se utilizaron para la extracción de datos de cada super. Parse es el método que recupera los datos y asigna valores a los distintos campos de Producto. Los métodos exacto, variable y todo son llamados luego de haber accedido a las urls que cumplen con las reglas dadas y si se cumple cada condicion solicitada realizan un yield donde recogen y almacenan los datos de la web.

En el caso particular del spider con name 'laamistad' tiene dos reglas ya que la pagina carga los producto de forma vertical y horizontal.

In [5]:
class LaamistadSpider(CrawlSpider):
    name = 'laamistad'
    busqueda = ''
    allowed_domains = ['tiendalaamistad.com.ar']

    def formatear(self,busqueda):
        busqueda = ''.join((c for c in unicodedata.normalize('NFD', busqueda) if unicodedata.category(c) != 'Mn'))
        for char in string.punctuation + "¡¿":
            busqueda = busqueda.replace(char, '')
        return busqueda.lower()

    def parse(self, response):
        producto = ItemLoader(Producto(), response)
        producto.add_value('supermercado', 'LA AMISTAD')
        producto.add_xpath('producto',
                           '/html/body/div[1]/div/div/div/main/div/div[1]/div[2]/div[1]/div/div[2]/div/h1/text()')
        producto.add_xpath('categoria',
                           '/html/body/div[1]/div/div/div/main/div/div[1]/div[2]/div[1]/div/div[2]/div/div[3]/span[2]/a[1]/text()')
        precio = response.xpath(

            '/html/body/div[1]/div/div/div/main/div/div[1]/div[2]/div[1]/div/div[2]/div/p[1]/span/text()').getall()[0]
        precio = '$' + precio
        producto.add_value('precio', precio)
        producto.add_value('link', response.url)
        producto.add_css('marca', '.woocommerce-product-attributes-item--attribute_pa_marca a::text')
        time = str(datetime.now().time().isoformat('seconds'))
        date = str(datetime.now().date())
        producto.add_value('fecha_de_extraccion', date)
        producto.add_value('hora', time)
        comparar = response.xpath(
            '/html/body/div[1]/div/div/div/main/div/div[1]/div[2]/div[1]/div/div[2]/div/h1/text()').getall()[0]
        comparar=self.formatear(comparar)
        return producto.load_item(), comparar

    def exacto(self, response):
        producto = self.parse(response)[0]
        resultado = self.parse(response)[1]
        if self.busqueda == resultado:
            yield producto

    def variable(self, response):
        producto = self.parse(response)[0]
        resultado = self.parse(response)[1].split(' ')
        contiene_alguna_palabra = False
        for palabra_busqueda in self.busqueda.split(' '):
            for palabra_resultado in resultado:
                if palabra_busqueda == palabra_resultado:
                    contiene_alguna_palabra = True
                    break
            if contiene_alguna_palabra:
                break
        if contiene_alguna_palabra:
            yield producto

    def todo(self, response):
        producto = self.parse(response)[0]
        resultado = self.parse(response)[1].split(" ")
        contiene_todo = True
        for palabra in self.busqueda.split(" "):
            if not resultado.__contains__(palabra):
                contiene_todo = False
                break
        if contiene_todo:
            yield producto

### <center>"Interfaz" de usuario.</center>
Aquí es donde se ejecuta el programa, solicitará ingresar una busqueda, determinar un formato y escribir un archivo ordenado en formato .csv
Si la búsqueda está vacía, no podrá buscar mas.


In [6]:
def main():
    aceptado = False
    print('¿Qué buscamos?')
    busqueda = BuscadorVariable.formatear(BuscadorVariable,str(input()))
    LaamistadSpider.busqueda = busqueda
    if busqueda!='':
        while not aceptado:
            print('¿En qué formato?\n1.CSV ordenado de menor a mayor.\n2.JSON crudo.')
            seleccion = int(input())
            if seleccion == 1:
                formato = 'csv'
                aceptado = True
            elif seleccion == 2:
                formato = 'json'
                aceptado = True
        aceptado = False
        while not aceptado:
            print('¿Qué tipo de busqueda?\n1.Publicación con la frase exacta'
                  '\n2.Publicación que contenga todas las palabras'
                  '\n3.Publicación que contenga algunas de las palabras')
            seleccion = int(input())
            if seleccion == 1:
                aceptado = True
                nuevo_buscador=BuscadorExacto(busqueda,formato)
                nuevo_buscador.iniciar_busqueda()
                if formato=='csv':
                    nuevo_buscador.generar_archivo_ordenado_por_menor_precio()
            elif seleccion == 3:
                aceptado = True
                nuevo_buscador=BuscadorVariable(busqueda,formato)
                nuevo_buscador.iniciar_busqueda()
                if formato == 'csv':
                    nuevo_buscador.generar_archivo_ordenado_por_menor_precio()
            elif seleccion == 2:
                aceptado = True
                nuevo_buscador=BuscadorTodo(busqueda,formato)
                nuevo_buscador.iniciar_busqueda()
                if formato == 'csv':
                    nuevo_buscador.generar_archivo_ordenado_por_menor_precio()
    else:
        print('No buscamos nada, chau!')
                
if __name__ == '__main__':
    main()

¿Qué buscamos?
harina integral
¿En qué formato?
1.CSV ordenado de menor a mayor.
2.JSON crudo.
1
¿Qué tipo de busqueda?
1.Publicación con la frase exacta
2.Publicación que contenga todas las palabras
3.Publicación que contenga algunas de las palabras
2


2020-12-03 11:32:26 [scrapy.utils.log] INFO: Scrapy 2.4.0 started (bot: scrapybot)
2020-12-03 11:32:26 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.5 (default, Jul 28 2020, 12:59:40) - [GCC 9.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1f  31 Mar 2020), cryptography 2.8, Platform Linux-5.4.0-54-generic-x86_64-with-glibc2.29
2020-12-03 11:32:26 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-12-03 11:32:26 [scrapy.crawler] INFO: Overridden settings:
{'FEED_EXPORT_ENCODING': 'utf-8',
 'FEED_EXPORT_FIELDS': ['fecha_de_extraccion',
                        'precio',
                        'producto',
                        'marca',
                        'supermercado',
                        'categoria',
                        'link',
                        'hora'],
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
               '(

2020-12-03 11:32:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://tiendalaamistad.com.ar/tienda/desayuno-y-merienda/galletitas/dulces-secas/cachafaz-225-gr-gall-harina-integral-y-algar/>
{'categoria': ['- DULCES SECAS'],
 'fecha_de_extraccion': ['2020-12-03'],
 'hora': ['11:32:34'],
 'link': ['https://tiendalaamistad.com.ar/tienda/desayuno-y-merienda/galletitas/dulces-secas/cachafaz-225-gr-gall-harina-integral-y-algar/'],
 'marca': ['CACHAFAZ'],
 'precio': ['$139.99'],
 'producto': ['CACHAFAZ 225 GR GALL HARINA INTEGRAL Y ALGAR'],
 'supermercado': ['LA AMISTAD']}
2020-12-03 11:32:35 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://tiendalaamistad.com.ar/tienda/almacen/harinas/harina-leudante/pureza-harina-leudante-1-kg/> (referer: https://tiendalaamistad.com.ar/page/2/?s=harina)
2020-12-03 11:32:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://tiendalaamistad.com.ar/tienda/almacen/harinas/harina-leudante/blancaflor-harina-leudante-n-1-kg/> (referer: https://tien